<a href="https://colab.research.google.com/github/drica-monteiro/intro_estat/blob/main/mcmc_def_int.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

As the previus notebook explored the integrals over the real line, now we want to compute definite integrals as $$\int_a^bh(x)f(x)dx.$$ Note that we can turn this integral into a integral over $\mathbb{R}$ w.r.t a distribution just by rewriting
$$\int_\mathbb{R}h(x)f(x)\cdot \mathbb{1}_{[a,b]}(x)dx = \alpha \cdot \int_\mathbb{R}h(x)P(x)dx$$
where $P(x) = \frac{f(x)}{\alpha} \mathbb{1}_{[a,b]} $ is a distribution. To turn $P$ a distribution, we need to guarantee that it integrates to one, that is,  
$$\alpha = \int_a^bf(x)dx.$$
The idea is that the preivous Metropolis Hastings still works when $f$ is not a distribution itself, but it is a distribution up to a constant. Then we can compute $\hat{I} = \int_\mathbb{R}h(x)f(x)\mathbb{1}_{[a,b]}(x)dx$ as before and plug in
$$\int_a^bh(x)f(x)dx = \alpha \hat{I} .$$

In [43]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm

The Cauchy distribution has density $f(x)=\frac{1}{\pi}\frac{1}{1+x^2}$. Our goal is to simulate a Markov chain whose stationary distribution is $f$.

Our choice for the proposal distribution is $q(y\mid x)=\mathcal{N}(x,b^2)$. Then, the acceptance rate is $r(x,y) = \min\left\{\frac{f(y)}{f(x)}, 1\right\} = \min\left\{\frac{1+x^2}{1+y^2}, 1\right\}$. The last step is to generate $U(0,1)\sim \text{Unif}(0,1)$ and if $U <r(x,y),$ set $X_{i+1} = Y$ otherwise $X_{i+1} = X_i$.

The normalization factionr $\alpha$ can be computed analitically.

In [ ]:
norm_fac = 0.648

In [ ]:
def cau_dens(x):
  if -1<=x<=3:
    dens = (np.pi*(1+x**2))**(-1)
  else:
    dens = 0
  return dens

In [ ]:
def accep_rate(func, x,y):
  f_x = func(x)
  f_y = func(y)
  rate = f_y/f_x
  if rate < 1:
    r = rate
  else:
    r = 1
  return r

In [44]:
def mcmc(func, n_steps = 100000, b = 1):
  x0 = 0
  xt = x0
  samples = []
  rates = []
  unif_samples = []
  for i in range(n_steps):
    unif_sample = np.random.uniform(0,1)
    yt = np.random.normal(xt, b)
    r = accep_rate(func, xt, yt)
    rates.append(r)
    unif_samples.append(unif_sample)
    if unif_sample < r:
      xt = yt
    samples.append(xt)

  burn_in = 1000
  samples = np.array(samples[burn_in:])
  return np.array(samples), np.array(rates), unif_samples


In [ ]:
####COMPUTING DEFINITE INTEGRAL

In [ ]:
def integrand(x):
  return x**2

We are computing $$\int_a^bx^2\frac{1}{\pi}\frac{1}{1+x^2}dx$$ using MCMC integration.

In [45]:
samples, rates, unif_samples = mcmc(cau_dens, b = 30)

In [46]:
I = norm_fac*np.mean(integrand(samples))
I

np.float64(0.6554465363834074)

Symbolab says that this integral is roughlly $≈0.62565$, so our approximation is good enough.

ATTENTION:
This class of algorithm is used to compute integrals UP TO A MULTIPLICATIVE CONSTANT. I mean, when you don't know how to compute the normalization factor and it is enough to get a computation of the integral up to a constant. So in practice, there is no way to compute `norm_fac`.